## 1. 製作各版本ipykernel 
<span style="color:red">Change to Default kernel:  Pyhon 3 (ipykernel)</span>.

In [ ]:
%%bash
# ipykernel
ml libs/singularity/3.10.2
singularity exec /work/u00cjz00/nvidia/python_3.8.sif pip install -q ipykernel
singularity exec /work/u00cjz00/nvidia/python_3.9.sif pip install -q ipykernel
singularity exec /work/u00cjz00/nvidia/python_3.10.sif pip install -q ipykernel
singularity exec /work/u00cjz00/nvidia/python_3.11.sif pip install -q ipykernel

In [ ]:
!ls  ~/.local/lib

## 2. 製作共用 library folder 之 image容器 kernel
<span style="color:red">Change to Default kernel:  Pyhon 3 (ipykernel)</span>.

In [ ]:
%%bash
### 1. IMAGE
IMAGE=/work/u00cjz00/nvidia/alpaca-lora_latest.sif
IMAGE_basename=G_$(basename "$IMAGE" .sif)

### 2. KERNEL SOURCE
IPYKERNEL=/work/u00cjz00/slurm_jobs/ipykernel/t2/image_with_ipykernel

### 3. KERNEL TARGET
mkdir -p ${HOME}/.local/share/jupyter/kernels/
rm -rf ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}
cp -rf ${IPYKERNEL} ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}
chmod -R 755 ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}

### 4. INSET IMAGE SOURCE and NAME in KERNEL.JSON 
IMAGE_desc=$(echo $IMAGE | sed 's_/_\\/_g')
sed -i "s/templateSIF/${IMAGE_desc}/g" ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json
sed -i "s/templateImage/Image_${IMAGE_basename}/g" ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json

In [ ]:
%%bash 
### CHECK  KERNEL.JSON 
IMAGE=/work/u00cjz00/nvidia/alpaca-lora_latest.sif
IMAGE_basename=G_$(basename "$IMAGE" .sif)
cat ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json

## TRY IT
1. Lauch  a new notebook tab
2. Change to kernel:  Image_G_alpaca-lora_latest
3. In first cell add content below

```
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
```

## 3. 製作專用 library folder 之 image容器 kernel
<span style="color:red">Change to Default kernel:  Pyhon 3 (ipykernel)</span>.

In [5]:
%%bash
### 0. pipi install libsray savefolder
saveFolder=home

### 1. IMAGE
IMAGE=/work/u00cjz00/nvidia/pytorch_2.1.0-cuda11.8-cudnn8-devel.sif
IMAGE_basename=S_${saveFolder}_$(basename "$IMAGE" .sif)

### 2. VIRTUAL LIBRARY and BINARY FOLDER
libraryFolder=/${saveFolder}/$(whoami)/libraryFolder/${IMAGE_basename}/local/lib
libraryFolder_binding=${libraryFolder}:${HOME}/.local/lib
binFolder=/${saveFolder}/$(whoami)/libraryFolder/${IMAGE_basename}/local/bin
binFolder_binding=${binFolder}:${HOME}/.local/bin
rm -rf /${saveFolder}/$(whoami)/libraryFolder/${IMAGE_basename}
mkdir -p ${libraryFolder} ${binFolder}

# 3. PIP INSTALL SLAVE IPYKERNEL
ml libs/singularity/3.10.2
python_version=$(singularity exec ${IMAGE} python3 -c 'import sys; print(".".join(map(str, sys.version_info[:2])))')
singularity exec -B ${libraryFolder_binding} -B ${binFolder_binding} /work/u00cjz00/nvidia/python_${python_version}.sif pip install -q ipykernel

# 4. IPYKERNEL for IMAGE
IPYKERNEL=/work/u00cjz00/slurm_jobs/ipykernel/t2/image_with_ipykernel_local
mkdir -p ${HOME}/.local/share/jupyter/kernels/
rm -rf ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}
cp -rf ${IPYKERNEL} ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}
chmod -R 755 ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}
IMAGE_desc=$(echo $IMAGE | sed 's_/_\\/_g')
sed -i "s/templateSIF/${IMAGE_desc}/g" ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json
sed -i "s/templateImage/Image_${IMAGE_basename}/g" ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json
sed -i "s@templateLibrayFolder@${libraryFolder_binding}@g" ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json
sed -i "s@templateBinFolder@${binFolder_binding}@g" ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
%%bash
### CHECK  KERNEL.JSON 
### 0. pipi install libsray savefolder
saveFolder=home
IMAGE=/work/u00cjz00/nvidia/pytorch_2.1.0-cuda11.8-cudnn8-devel.sif
IMAGE_basename=S_${saveFolder}_$(basename "$IMAGE" .sif)
cat /home/$(whoami)/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json

{
 "language": "python",
 "argv": ["/work/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity",
   "exec",
   "--nv",
   "--cleanenv",
   "-B",
   "/work",
   "-B",
   "/home/g00cjz00/libraryFolder/S_home_pytorch_2.1.0-cuda11.8-cudnn8-devel/local/lib:/home/g00cjz00/.local/lib",
   "-B",
   "/home/g00cjz00/libraryFolder/S_home_pytorch_2.1.0-cuda11.8-cudnn8-devel/local/bin:/home/g00cjz00/.local/bin",
   "-B",
   "/work/u00cjz00/os/ubuntu/bin:/usr/ubuntu_bin",
   "/work/u00cjz00/nvidia/pytorch_2.1.0-cuda11.8-cudnn8-devel.sif",
   "python3",
   "-m",
   "ipykernel",
   "-f",
   "{connection_file}"
 ],
 "display_name": "Image_S_home_pytorch_2.1.0-cuda11.8-cudnn8-devel"
}


## TRY IT
1. Lauch  a new notebook tab
2. Change to kernel:  Image_S_home_pytorch_2.1.0-cuda11.8-cudnn8-devel
3. In first cell add content below

```
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
```

## 4. DEMO: Image容器
<span style="color:red">Change to Default kernel:  Image_S_home_pytorch_2.1.0-cuda11.8-cudnn8-devel</span>.

In [1]:
# 新增讀取套件安裝位置
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [2]:
!pip install chainlit -q
!pip list -v |grep chainlit

chainlit                                 0.7.604      /home/g00cjz00/.local/lib/python3.10/site-packages pip


In [3]:
!chainlit --help

Usage: chainlit [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  create-secret
  hello
  init
  run


In [4]:
!which chainlit

/home/g00cjz00/.local/bin/chainlit


In [2]:
!ls $HOME/libraryFolder/S_home_pytorch_2.1.0-cuda11.8-cudnn8-devel/local/bin

chainlit  ipython3	      jupyter-run		uvicorn
dotenv	  jupyter	      jupyter-troubleshoot	watchfiles
filetype  jupyter-kernel      opentelemetry-bootstrap
httpx	  jupyter-kernelspec  opentelemetry-instrument
ipython   jupyter-migrate     pygmentize


## 5. Singularity指令 + 共用 library folder 之 image容器 kernel
<span style="color:red">Change to Default kernel:  Pyhon 3 (ipykernel)</span>.

In [ ]:
%%bash
### 1. IMAGE
IMAGE=/work/u00cjz00/nvidia/alpaca-lora_latest.sif
IMAGE_basename=S_$(basename "$IMAGE" .sif)

# 2. SINGULARITY 
ml libs/singularity/3.10.2
singularity exec --nv -B /work ${IMAGE} \
bash -c "export PATH=$PATH:$HOME/.local/bin:/work/u00cjz00/os/ubuntu/bin; git --help"

## 6. Singularity指令 + 專用 library folder 之 image容器 kernel
<span style="color:red">Change to Default kernel:  Pyhon 3 (ipykernel)</span>.

In [ ]:
%%bash
### 1. IMAGE
IMAGE=/work/u00cjz00/nvidia/alpaca-lora_latest.sif
IMAGE_basename=S_$(basename "$IMAGE" .sif)

### 2. VIRTUAL LIBRARY and BINARY FOLDER
libraryFolder=/home/$(whoami)/libraryFolder/${IMAGE_basename}/.local/lib
libraryFolder_binding=${libraryFolder}:/home/$(whoami)/.local/lib
binFolder=/home/$(whoami)/libraryFolder/${IMAGE_basename}/.local/bin
binFolder_binding=${binFolder}:/home/$(whoami)/.local/bin

# 3. SINGULARITY 
ml libs/singularity/3.10.2
singularity exec -B ${libraryFolder}:/home/$(whoami)/.local/lib -B ${binFolder}:/home/$(whoami)/.local/bin ${IMAGE} \
bash -c "export PATH=$PATH:$HOME/.local/bin:/work/u00cjz00/os/ubuntu/bin; git --help"

## 7. Slurm工作排程 + Singularity指令 + 專用 library folder 之 image容器 kernel
<span style="color:red">ONLY DEMO CODE</span>.

## 8. EOF + Singularity指令 + 專用 library folder 之 image容器 kernel
<span style="color:red">Change to Default kernel:  Pyhon 3 (ipykernel)</span>.

In [ ]:
%%bash
cat << \EOF > job.sh
### 1. IMAGE
IMAGE=/work/u00cjz00/nvidia/alpaca-lora_latest.sif
IMAGE_basename=S_$(basename "$IMAGE" .sif)

### 2. VIRTUAL LIBRARY and BINARY FOLDER
libraryFolder=/home/$(whoami)/libraryFolder/${IMAGE_basename}/.local/lib
libraryFolder_binding=${libraryFolder}:/home/$(whoami)/.local/lib
binFolder=/home/$(whoami)/libraryFolder/${IMAGE_basename}/.local/bin
binFolder_binding=${binFolder}:/home/$(whoami)/.local/bin

# 3. SINGULARITY 
ml libs/singularity/3.10.2
singularity exec --nv -B /work -B ${libraryFolder}:/home/$(whoami)/.local/lib -B ${binFolder}:/home/$(whoami)/.local/bin ${IMAGE} \
bash -c "export PATH=$PATH:$HOME/.local/bin:/work/u00cjz00/os/ubuntu/bin; git --help"
EOF

In [ ]:
!cat job.sh

In [ ]:
!bash job.sh

## 9. EOF + Slurm工作排程 + Singularity指令 + 專用 library folder 之 image容器 kernel
<span style="color:red">Change to Default kernel:  Pyhon 3 (ipykernel)</span>.

In [ ]:
%%bash
cat << \EOF > job.slurm
#!/work/u00cjz00/binary/bash5.0/bin/bash
#SBATCH -A MST110386                                                    ### project number, Example MST109178
#SBATCH -J _t2lora                                                      ### Job name, Exmaple jupyterlab
#SBATCH -p gp4d                                                         ### Partition Name, Example ngs1gpu
#SBATCH --nodes=1                                                       ### Nodes, Default 1, node number
#SBATCH --ntasks-per-node=1                                             ### Tasks, Default 1, per node tasks
#SBATCH -c 4                                                            ### Cores assigned to each task, Example 4
#SBATCH --gres=gpu:1                                                    ### GPU number, Example gpu:1
#SBATCH --time=0-1:00:00                                                ### Runnung time, days-hours:minutes:seconds or hours:minutes:seconds
#SBATCH -o genai_%j.out                                                 ### Log folder, Here %j is job ID
#SBATCH -e genai_%j.err                                                 ### Log folder, Here %j is job ID

### 1. IMAGE
IMAGE=/work/u00cjz00/nvidia/alpaca-lora_latest.sif
IMAGE_basename=S_$(basename "$IMAGE" .sif)

### 2. VIRTUAL LIBRARY and BINARY FOLDER
libraryFolder=/home/$(whoami)/libraryFolder/${IMAGE_basename}/.local/lib
libraryFolder_binding=${libraryFolder}:/home/$(whoami)/.local/lib
binFolder=/home/$(whoami)/libraryFolder/${IMAGE_basename}/.local/bin
binFolder_binding=${binFolder}:/home/$(whoami)/.local/bin

# 3. SINGULARITY 
ml libs/singularity/3.10.2
singularity exec -B ${libraryFolder}:/home/$(whoami)/.local/lib -B ${binFolder}:/home/$(whoami)/.local/bin ${IMAGE} \
bash -c "export PATH=$PATH:$HOME/.local/bin:/work/u00cjz00/os/ubuntu/bin; git --help; sleep 30"
pid0=$!

# 4. Computing NODE
noed_hostname=$(hostname -s)
noed_port=$(python3 /work/u00cjz00/binary/availablePort.py)
node_ip=$(cat /etc/hosts |grep "$(hostname -a)" | awk '{print $1}')

# 5. Output message
echo ""
echo ""
echo "****************************** 請輸入下方指令 ******************************"
echo ""
echo "# STEP1: Execute cmd in your client below to login login_node"
echo ssh ${iam}@ln01.twcc.ai
echo ""
echo "# STEP2: Login  computing_node "
echo ssh ${noed_hostname}
echo ""
echo ""
echo "***********************************************************************************************"

## 6. Get pid
pid0=$!
sleep 2
pid1=$(pgrep -P ${pid0})
pid=${pid1}
echo "PID0:${pid0} , PID:${pid1}"

## 7. WAIT JOB
wait $pid0

EOF

In [ ]:
!cat job.slurm

In [ ]:
!sbatch job.slurm